# Libs

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from lightgbm import LGBMRegressor
import optuna

# Params

In [2]:
PATH_TRAIN = 'train_data.csv'
PATH_TEST = 'test_data.csv'

# EDA

In [3]:
df_train = pd.read_csv(PATH_TRAIN)
df_train.head()

,eventTimestamp,unitDisplayType,brandName,bundleId,appVersion,correctModelName,countryCode,deviceId,osAndVersion,connectionType,c1,c2,c3,c4,size,mediationProviderVersion,bidFloorPrice,sentPrice,winBid,has_won
0,1656411567773,banner,LG,com.tilegarden.match3,1.8.22,LM-V405,US,74f9b473fad,Android-10.0,3G,cb2,6.0,6b,4.0,320x50,11.2.1,0.01,0.02,0.88,0
1,1656925395488,banner,Generic,com.loop.match3d,1245.34.0,Android 4.0,FR,6ad4c88b84e,Android-4.0,WIFI,7d3,6.0,6b,6.0,320x50,11.4.2,0.01,0.03,0.08,0
2,1656913751642,banner,Generic,com.loop.match3d,1245.35.0,Android 4.0,US,743b9849642,Android-4.0,WIFI,7d3,3.0,6b,3.0,320x50,11.4.3,0.01,0.02,1.72,0
3,1656656319103,banner,Generic,com.loop.match3d,1245.34.0,Android 4.0,GB,6ad933115b2,Android-4.0,WIFI,7d3,3.0,6b,5.0,320x50,11.4.2,0.01,0.06,0.21,0
4,1657429389462,interstitial,OPPO,com.loop.match3d,1245.35.0,CPH2127,ID,809f9785bb3,Android-11.0,WIFI,8bd,3.0,6b,3.0,320x480,11.4.3,0.01,0.16,1.91,0


In [4]:
df_test = pd.read_csv(PATH_TEST)
df_test.head()

,eventTimestamp,unitDisplayType,brandName,bundleId,appVersion,correctModelName,countryCode,deviceId,osAndVersion,connectionType,c1,c2,c3,c4,size,mediationProviderVersion,bidFloorPrice,sentPrice
0,1657758857892,rewarded,Generic,com.loop.match3d,1245.35.0,Android 4 Tablet,US,6a0a94554cf,Android-4.0,3G,7b8,2.0,6b,2.0,768x1024,11.4.3,0.05,0.06
1,1657210707978,interstitial,Generic,com.loop.match3d,1245.35.0,Android 4.0,ZA,6a0b0e59f45,Android-4.0,WIFI,8bd,8.0,6b,6.0,320x480,11.4.3,0.01,0.16
2,1657392939412,interstitial,Motorola,com.YayySAL.DodgeAgent,13.0.02,Moto g(8) power lite,BR,6a0fa820c46,Android-10.0,WIFI,ea0,8.0,6b,9.0,320x480,11.0.0,0.05,0.06
3,1657386816882,interstitial,Huawei,com.tintash.nailsalon,1.3.6,JDN2-AL00HN,IQ,6a142bdbea2,Android-9.0,WIFI,3dc,6.0,6b,4.0,768x1024,11.3.3,0.01,0.05
4,1657211600823,banner,Generic,com.tilegarden.match3,1.8.52,Android 4.0,US,6a16943a771,Android-4.0,WIFI,cb2,7.0,6b,5.0,320x50,11.4.3,0.01,0.03


In [5]:
df_train.describe()

,eventTimestamp,c2,c4,bidFloorPrice,sentPrice,winBid,has_won
count,7.321633e+06,7.321633e+06,7.321633e+06,7.321633e+06,7.321633e+06,7.321633e+06,7.321633e+06
mean,1.657045e+12,5.000590e+00,4.999044e+00,1.752807e-02,5.810363e-01,4.184625e+00,1.109695e-01
std,3.964243e+08,2.344387e+00,2.345298e+00,1.316725e-01,3.807544e+00,2.069451e+01,3.140944e-01
min,1.656290e+12,1.000000e+00,1.000000e+00,1.000000e-02,1.000000e-02,1.000000e-02,0.000000e+00
25%,1.656710e+12,3.000000e+00,3.000000e+00,1.000000e-02,2.000000e-02,9.000000e-02,0.000000e+00
50%,1.657043e+12,5.000000e+00,5.000000e+00,1.000000e-02,4.000000e-02,5.100000e-01,0.000000e+00
75%,1.657388e+12,7.000000e+00,7.000000e+00,1.000000e-02,1.700000e-01,1.540000e+00,0.000000e+00
max,1.657759e+12,9.000000e+00,9.000000e+00,5.000000e+00,9.704100e+02,3.405720e+03,1.000000e+00


In [6]:
df_train.shape

(7321633, 20)

## Timestamp to Y M D H

In [7]:
def ymdh(df):
    df = df.copy()
    df['eventTimestamp'] = pd.to_datetime(df['eventTimestamp'], unit='ms')
    
    df['Year'] = df['eventTimestamp'].dt.year
    df['Month'] = df['eventTimestamp'].dt.month
    df['DayOfWeek'] = df['eventTimestamp'].dt.day_name()
    df['Hour'] = df['eventTimestamp'].dt.hour
    
    return df

In [8]:
df_train = ymdh(df_train)
df_test = ymdh(df_test)

In [9]:
df_train.head()

,eventTimestamp,unitDisplayType,brandName,bundleId,appVersion,correctModelName,countryCode,deviceId,osAndVersion,connectionType,c1,c2,c3,c4,size,mediationProviderVersion,bidFloorPrice,sentPrice,winBid,has_won,Year,Month,DayOfWeek,Hour
0,2022-06-28 10:19:27.773,banner,LG,com.tilegarden.match3,1.8.22,LM-V405,US,74f9b473fad,Android-10.0,3G,cb2,6.0,6b,4.0,320x50,11.2.1,0.01,0.02,0.88,0,2022,6,Tuesday,10
1,2022-07-04 09:03:15.488,banner,Generic,com.loop.match3d,1245.34.0,Android 4.0,FR,6ad4c88b84e,Android-4.0,WIFI,7d3,6.0,6b,6.0,320x50,11.4.2,0.01,0.03,0.08,0,2022,7,Monday,9
2,2022-07-04 05:49:11.642,banner,Generic,com.loop.match3d,1245.35.0,Android 4.0,US,743b9849642,Android-4.0,WIFI,7d3,3.0,6b,3.0,320x50,11.4.3,0.01,0.02,1.72,0,2022,7,Monday,5
3,2022-07-01 06:18:39.103,banner,Generic,com.loop.match3d,1245.34.0,Android 4.0,GB,6ad933115b2,Android-4.0,WIFI,7d3,3.0,6b,5.0,320x50,11.4.2,0.01,0.06,0.21,0,2022,7,Friday,6
4,2022-07-10 05:03:09.462,interstitial,OPPO,com.loop.match3d,1245.35.0,CPH2127,ID,809f9785bb3,Android-11.0,WIFI,8bd,3.0,6b,3.0,320x480,11.4.3,0.01,0.16,1.91,0,2022,7,Sunday,5


## NA values

In [10]:
df_train.isna().sum()

eventTimestamp                0
unitDisplayType               0
brandName                     0
bundleId                      0
appVersion                    0
correctModelName              0
countryCode                  27
deviceId                      0
osAndVersion                  0
connectionType              666
c1                            0
c2                            0
c3                            0
c4                            0
size                          0
mediationProviderVersion      0
bidFloorPrice                 0
sentPrice                     0
winBid                        0
has_won                       0
Year                          0
Month                         0
DayOfWeek                     0
Hour                          0
dtype: int64

In [11]:
df_test.isna().sum()

eventTimestamp               0
unitDisplayType              0
brandName                    0
bundleId                     0
appVersion                   0
correctModelName             0
countryCode                  6
deviceId                     0
osAndVersion                 0
connectionType              17
c1                           0
c2                           0
c3                           0
c4                           0
size                         0
mediationProviderVersion     0
bidFloorPrice                0
sentPrice                    0
Year                         0
Month                        0
DayOfWeek                    0
Hour                         0
dtype: int64

In [12]:
df_train.connectionType.value_counts()

connectionType
WIFI       5844929
3G         1168337
UNKNOWN     307701
Name: count, dtype: int64

In [13]:
#Not many NAs in the dataset.
#Filling in NA connection types with 'unknown' value since it already exists in the data.
df_train.loc[df_train.connectionType.isna(), 'connectionType'] = 'UNKNOWN'
df_test.loc[df_test.connectionType.isna(), 'connectionType'] = 'UNKNOWN'

In [14]:
#Filling the nan country codes with 'N/' which seems to be the 'unknown' equivalent existing in the data
df_train.loc[df_train.countryCode.isna(), 'countryCode'] = 'N/'
df_test.loc[df_test.countryCode.isna(), 'countryCode'] = 'N/'

## Distributions

In [15]:
def plot_vc(df, col):    
    val_cnt = df[col].value_counts(normalize=True).reset_index()  
    val_cnt.columns = [col, 'proportion']
    val_cnt = val_cnt.sort_values('proportion', ascending=False)     
    
    fig = px.bar(val_cnt, x=col, y='proportion', title=f'Distribution of {col} Values',
                 labels={'proportion': 'Proportion'})      

    fig.update_layout(
        xaxis_title=col,
        yaxis_title="Proportion",
        xaxis={'categoryorder': 'total descending'}  
    )    

    fig.show()

In [16]:
plot_vc(df_train, 'unitDisplayType')

In [17]:
plot_vc(df_train, 'brandName')

In [18]:
plot_vc(df_train, 'bundleId')

In [19]:
plot_vc(df_train, 'appVersion')

In [20]:
plot_vc(df_train, 'correctModelName')

In [21]:
plot_vc(df_train, 'countryCode')

In [22]:
plot_vc(df_train, 'connectionType')

In [23]:
plot_vc(df_train, 'c1')

In [24]:
plot_vc(df_train, 'c2')

In [25]:
plot_vc(df_train, 'c3')

In [26]:
plot_vc(df_train, 'c4')

In [27]:
plot_vc(df_train, 'size')

In [28]:
plot_vc(df_train, 'has_won')

In [ ]:
fig = px.histogram(df_train, x="winBid")
fig.show()

## Cap target outliers

In [30]:
def cap_outliers(df, column_name, percentile=0.95):
    cap_value = df[column_name].quantile(percentile)
    df[column_name] = df[column_name].clip(upper=cap_value)    
    return df

df_train = cap_outliers(df_train, 'winBid', 0.95)

In [ ]:
fig = px.histogram(df_train, x="winBid")
fig.show()

# Target by feature

In [32]:
feats = ['unitDisplayType', 'bundleId', 'connectionType', 'c1','c2', 'c3', 'c4', 'size', 'DayOfWeek', 'Month', 'Hour', 'brandName', 'correctModelName', 'countryCode', 'osAndVersion', 'mediationProviderVersion', 'appVersion']
for feat in feats:
    fig = px.bar(df_train.groupby(feat, as_index=False)['winBid'].mean(), x=feat, y='winBid', title=f'Avg winBid by {feat}')
    fig.show()

# Transf

## Device history

In [33]:
#Adding lag features for 5 last won bids

#Combining train and test to add lag features to test as well
df_test['has_won'] = np.nan
df_test['winBid'] = np.nan

df_test['source'] = 'test'
df_train['source'] = 'train'

df_combined = pd.concat([df_train, df_test])


df_combined.sort_values(by = ['deviceId','eventTimestamp'], inplace = True)
gb = df_combined.groupby(['deviceId','size'], as_index = False)

for i in range(1,6):
    col = f'sft_{i}'
    df_combined[col] = gb['winBid'].shift(i)    

In [34]:
df_test = df_combined[df_combined.source == 'test']
df_train = df_combined[df_combined.source == 'train']

## Dropping / combining features

In [35]:
#Note - since LGBM will be used there is no need to drop below features.
# #Dropping features with no impact on target and/or too many cat values
# df_train = df_train.drop(columns=['c2', 'c4', 'DayOfWeek', 'Month', 'correctModelName'])



#Note - high cardinality will be handled by LGBM without the need to transform.
# #Combining software features
# df_train['mediationProviderVersion'] = df_train['mediationProviderVersion'].str[:2]
# fig = px.bar(df_train.groupby('mediationProviderVersion', as_index=False)['winBid'].mean(), x='mediationProviderVersion', y='winBid', title=f'Avg winBid by {'mediationProviderVersion'}')
# fig.show()

# df_train['appVersion'] = df_train['appVersion'].str.split('.').str[0]
# fig = px.bar(df_train.groupby('appVersion', as_index=False)['winBid'].mean(), x='appVersion', y='winBid', title=f'Avg winBid by {'appVersion'}')
# fig.show()

## Encoding

In [36]:
#Encoding not required due to LGBM handling them more efficiently.

# df_train_formatted = df_train.copy().reset_index(drop=True).drop(columns='source').fillna(0)
# exclude_columns = ['eventTimestamp', 'deviceId']
# non_numerical_columns = df_train_formatted.select_dtypes(exclude=[np.number]).columns.difference(exclude_columns)
# df_dummies = pd.get_dummies(df_train_formatted, columns=non_numerical_columns, drop_first=False)
# del df_train_formatted
# df_dummies.head()

In [37]:
# X = df_dummies.drop(columns=['winBid', 'has_won'])
# y = df_dummies[['eventTimestamp', 'deviceId', 'winBid', 'has_won']]
# del df_dummies
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [38]:
# #preserving IDs
# id_cols = ['eventTimestamp', 'deviceId']
# X_train_ids = X_train[id_cols]
# X_test_ids = X_test[id_cols]
# y_train_ids = y_train[id_cols]
# y_test_ids = y_test[id_cols]

# X_train = X_train.drop(columns=id_cols)
# X_test = X_test.drop(columns=id_cols)

# y_train = y_train.winBid
# y_test = y_test.winBid

# Model

## Split

In [39]:
df_train = df_train.reset_index(drop=True).drop(columns='source').fillna(0)

# Encoding categorical columns for LGBM
categorical_features = ['unitDisplayType', 'brandName', 'bundleId',
       'appVersion', 'correctModelName', 'countryCode',
       'osAndVersion', 'connectionType', 'c1',  'c3',  'size',
       'mediationProviderVersion', 'Month', 'DayOfWeek', 'Hour']
for col in categorical_features:
    df_train[col] = df_train[col].astype('category')

In [40]:
X = df_train.drop(columns=['winBid', 'has_won'])
y = df_train[['eventTimestamp', 'deviceId', 'winBid']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Preserve IDs

In [41]:
#preserving IDs
id_cols = ['eventTimestamp', 'deviceId']

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

X_train_ids = X_train[id_cols].copy()
X_test_ids = X_test[id_cols].copy()
y_train_ids = y_train[id_cols].copy()
y_test_ids = y_test[id_cols].copy()

X_train = X_train.drop(columns=id_cols)
X_test = X_test.drop(columns=id_cols)

y_train = y_train.winBid
y_test = y_test.winBid

## Initial Model

In [42]:
#Model
model = LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=100)
model.fit(X_train, y_train)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.153828 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3113
[LightGBM] [Info] Number of data points in the train set: 4905494, number of used features: 25
[LightGBM] [Info] Start training from score 2.095602


LGBMRegressor(learning_rate=0.05)

In [43]:
#Predict
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE on test data: {rmse}")

RMSE on test data: 1.049343245161282


In [44]:
print(f"MAPE on test data: {mean_absolute_percentage_error(y_test, y_pred)}")

MAPE on test data: 1.081075392837321


In [45]:
comparison_df = pd.DataFrame()
comparison_df['pred'] = y_pred
comparison_df['true'] = y_test
comparison_df.head(15)

,pred,true
0,0.406853,0.55
1,15.626247,11.19
2,1.844179,1.66
3,0.132550,0.03
4,0.095245,0.07
5,16.055003,15.92
6,6.103237,4.99
7,0.135041,0.10
8,9.994943,11.19
9,0.073236,0.01


## Hyperparameter optimization

In [46]:
# Define the objective function for Optuna
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'l2',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        #'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True), #deprecation warning
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }
    
    model = LGBMRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

# Create a study object
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

# Best parameters found
best_params = study.best_params
print(f"Best parameters: {best_params}")

[I 2024-05-08 11:21:49,517] A new study created in memory with name: no-name-60c10e78-6636-41a4-8e7b-64f4eb0284d9
[I 2024-05-08 11:22:24,557] Trial 0 finished with value: 1.0406070136581425 and parameters: {'num_leaves': 36, 'learning_rate': 0.02559950470104347, 'n_estimators': 240, 'max_depth': 8, 'min_child_samples': 92, 'subsample': 0.9357255651220162, 'colsample_bytree': 0.5766282849347835}. Best is trial 0 with value: 1.0406070136581425.
[I 2024-05-08 11:22:45,609] Trial 1 finished with value: 1.0348110831684962 and parameters: {'num_leaves': 36, 'learning_rate': 0.07531730457405185, 'n_estimators': 162, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.7815479197781272, 'colsample_bytree': 0.523398930900353}. Best is trial 1 with value: 1.0348110831684962.
[I 2024-05-08 11:23:33,279] Trial 2 finished with value: 1.0244661161377164 and parameters: {'num_leaves': 70, 'learning_rate': 0.031722315571978084, 'n_estimators': 292, 'max_depth': 10, 'min_child_samples': 96, 'subsamp

Best parameters: {'num_leaves': 96, 'learning_rate': 0.11991538855040626, 'n_estimators': 270, 'max_depth': 17, 'min_child_samples': 83, 'subsample': 0.9016119675454399, 'colsample_bytree': 0.7317904436685324}


In [47]:
# Train the final model with the best parameters
final_model = LGBMRegressor(**best_params)
final_model.fit(X_train, y_train)
final_predictions = final_model.predict(X_test)
final_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))

print(f"Final RMSE on test data: {final_rmse}")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.287629 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3111
[LightGBM] [Info] Number of data points in the train set: 4905494, number of used features: 24
[LightGBM] [Info] Start training from score 2.095602
Final RMSE on test data: 1.0123437871009502


In [48]:
print(f"Final MAPE on test data: {mean_absolute_percentage_error(y_test, final_predictions)}")

Final MAPE on test data: 0.7228458195895333


In [49]:
comparison_df = pd.DataFrame()
comparison_df['pred'] = final_predictions
comparison_df['true'] = y_test
comparison_df.head(15)

,pred,true
0,0.246560,0.55
1,15.471070,11.19
2,1.932155,1.66
3,0.093596,0.03
4,0.064039,0.07
5,15.758104,15.92
6,7.391927,4.99
7,0.150622,0.10
8,9.548041,11.19
9,0.017424,0.01


## Final prediction

In [50]:
df_test = df_test.reset_index(drop=True).drop(columns=['source', 'winBid', 'has_won']).fillna(0)
for col in categorical_features:
    df_test[col] = df_test[col].astype('category')

df_test_ids = df_test[id_cols].copy()
df_test_X = df_test.drop(columns=id_cols)

In [51]:
fin_pred = final_model.predict(df_test_X)
fin_pred_df = df_test_ids.copy()
fin_pred_df['prediction'] = fin_pred

fin_pred_df.head(15)

,eventTimestamp,deviceId,prediction
0,2022-07-04 14:46:16.303,6a000c885b8,0.206602
1,2022-07-09 06:37:46.486,6a0019a885a,0.121555
2,2022-07-12 21:41:23.784,6a00259e2d1,0.769698
3,2022-06-29 09:48:56.328,6a0025aa282,0.412932
4,2022-07-13 08:27:17.122,6a002954729,0.946311
5,2022-06-30 03:31:51.958,6a002c67d32,0.935096
6,2022-06-30 01:02:26.467,6a0031157a5,0.143038
7,2022-06-28 00:56:39.079,6a003a11b28,0.069440
8,2022-07-09 15:39:05.476,6a003bb8db2,0.315425
9,2022-06-29 18:55:21.932,6a00432ac05,0.166760


In [52]:
# Save submission
fin_pred_df.to_csv('prediction_submission.csv', index=False)